In [1]:
# libraries
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
from scipy.stats import pearsonr, kendalltau, spearmanr

# 2. Compute Stocks Prices Correlation

In this notebook we compute the correlation between all possible stock pairs over a moving time frame.

First, we create a DataFrame of log returns for all stocks (calculated on Adjusted Close):

In [2]:
# path to prices data
path = '../data/prices/'

# create empty dataframe (only business days)
data = pd.DataFrame(index=pd.date_range(start="1990-01-01", end="2021-01-01", freq='B'))

# iterate over files and add columns
files = os.listdir(path)
for file in files: 
    
    # import, keep log returns and change column name
    price = pd.read_csv(os.path.join(path, file), index_col=0)
    price = price[['LogRet_AdjClose']]
    price.columns = [file.replace('.csv', '')]
    
    # merge
    data  = pd.merge(data, price, left_index=True, right_index=True, how='outer')
    
data.head()

,CSCO,UAL,TROW,ISRG,NVR,PRGO,TPR,DVN,CE,MRO,...,CRM,PGR,WAT,IEX,BWA,LRCX,NWL,UAA,BLK,PPL
1990-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-01-03,NaN,NaN,0.016529,NaN,0.000000,NaN,NaN,0.018692,NaN,-0.006969,...,NaN,0.006472,NaN,0.000000,NaN,-0.022473,-0.005222,NaN,NaN,0.002903
1990-01-04,NaN,NaN,0.024293,NaN,0.024098,NaN,NaN,0.000000,NaN,0.024182,...,NaN,-0.006472,NaN,-0.029852,NaN,-0.095310,-0.005250,NaN,NaN,-0.008734
1990-01-05,NaN,NaN,-0.008033,NaN,0.023530,NaN,NaN,-0.018692,NaN,-0.013746,...,NaN,0.003242,NaN,-0.007604,NaN,0.000000,0.000000,NaN,NaN,-0.008810


Then, we define a function to compute pairwise correlations between stocks. In particular, we consider three possible different types of correlation: 
- [Pearson](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html)
- [Kendall](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kendalltau.html)
- [Spearman](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html)

For Pearson (the default type of correlation we use), we define a fastest implementation than the one provided in $scipy$ (more details [here](https://cancerdatascience.org/blog/posts/pearson-correlation/)):



In [3]:
def np_pearson_corr(x, y):
    
    """
    Fast implementation of Pearson correlation coefficient 
        :param x, y (numpy arrays): arrays on which we compute correlation
        :return: returns the Pearson corr. coeff. between x and y
    """
    
    xv, yv = x - x.mean(axis=0), y - y.mean(axis=0)
    xvss, yvss = (xv * xv).sum(axis=0), (yv * yv).sum(axis=0)
    result = np.matmul(xv.transpose(), yv) / np.sqrt(np.outer(xvss, yvss))
    # bound the values to -1 to 1 in the event of precision issues
    return np.maximum(np.minimum(result, 1.0), -1.0)[0][0]

We also have to deal with nan data. Indeed, some stocks start to be traded only after the beginning of our dataset (for example new companies), some others stop to be traded at a certain point (for example failed companies), and of course there can missing data here and there for other reasons. Therefore, we define a threshold of minimum non-nan values required in order to compute the correlation (otherwise we return a 0 correlation). We note that, since we are computing pairwise correlations, we need both prices to be non-nan at the same time.

In [4]:
def correlation(data_np, i, j, start, end, window, coeff="pearson", notnan_fraction=0.9):
    
    """
    This function computes the correlation between the prices of two stocks
    over a given a time period:
        :param data_np (numpy 2D-array): 2D-array of stocks prices
        :param i (int): index of ticker of first stock
        :param j (int): index of ticker of second stock
        :param start (int): start of time window (included)
        :param end (int): end of time window (excluded)
        :param coeff (string, default="pearson"): type of correlation (possible values: 'pearson', 'kendall', 'spearman')
        :param notnan_fracton (float, default=0.9): fraction of non-nan prices required in order to compute correlation
        :return: coefficient of correlation between two price series
    """
    
    # select data between start/end and get the two price time series
    price1 = data_np[start:end, i]
    price2 = data_np[start:end, j]
    
    # control nan values: if the fraction of non-nan pairs is smaller than the parameter
    # notnan_fraction we return a nan correlation (i.e. not enough data to compute it)
    # We define a pair of prices a 'nan-pair' if on a day one or both of the two prices is nan
    notnans = ~np.logical_or(np.isnan(price1), np.isnan(price2))
    
    # if enough non-nan data compute correlation
    if (notnans).sum() / window >= notnan_fraction:
        
        # compute correlation coefficient only on non-nan pairs (otherwise scipy raises an error)
        if coeff=="pearson": 
            return np_pearson_corr(np.compress(notnans, price1), np.compress(notnans, price2))

        elif coeff=="kendall": 
            return kendalltau(np.compress(notnans, price1), np.compress(notnans, price2))[0]

        elif coeff=="spearman":
            return spearmanr(np.compress(notnans, price1), np.compress(notnans, price2))[0]

        else: 
            print("Coefficient name not recognised. Possible values: 'pearson', 'kendall', 'spearman'")
    
    # not enough non-nan data, return 0 correlation
    else: 
        return 0

Finally, we compute correlation matrices (i.e matrices of correlation between all possible stocks) over a moving time frame. To keep data comparable at different time steps, we will keep rows and columns of the correlation matrix ordered like the columns of the stocks Pandas DataFrame (we save a file with the ordering in order to match row and column in next notebooks):

In [5]:
pd.DataFrame(data.columns, columns=["ticker"]).to_csv("../data/tickers_order.csv", index=False)

We define the moving time frame to be 3 months long and moving forward in steps of 1 month. Let's compute the correlation matrices:

In [6]:
# list of tickers
tickers = data.columns.values  

# numpy version of DataFrame (faster)
data_np = data.to_numpy()

start = 0
end   = data_np.shape[0]
current_step = start

# datetimes
current_date = data.index[0]

window = 90  # days
step   = 30  # days

# move the time window until end of observations
while current_step + window < end:
    
    # show advancement
    print(data.index[current_step])
    
    # initialize correlation matrix for this frame
    corr_matrix = np.zeros((tickers.shape[0], tickers.shape[0]), dtype=np.float16)
    
    # iterate over tickers to compute correlations
    for i in range(tickers.shape[0]):
        for j in range(tickers.shape[0]):
            if i > j:  # no self-correlation and matrix is symmetric
                corr_matrix[i, j] = correlation(data_np, i, j, current_step, current_step + window, window)
                
    # save correlation matrix
    file_name = current_date.strftime("%Y%m%d") + "_" + (current_date + timedelta(days=window)).strftime("%Y%m%d") + ".npz"
    np.savez_compressed("../data/corr_matrices/" + file_name, corr_matrix)

    # advance window
    current_date += timedelta(days=step)
    current_step += step

1990-01-01 00:00:00
1990-02-12 00:00:00
1990-03-26 00:00:00
1990-05-07 00:00:00
1990-06-18 00:00:00
1990-07-30 00:00:00
1990-09-10 00:00:00
1990-10-22 00:00:00
1990-12-03 00:00:00
1991-01-14 00:00:00
1991-02-25 00:00:00
1991-04-08 00:00:00
1991-05-20 00:00:00
1991-07-01 00:00:00
1991-08-12 00:00:00
1991-09-23 00:00:00
1991-11-04 00:00:00
1991-12-16 00:00:00
1992-01-27 00:00:00
1992-03-09 00:00:00
1992-04-20 00:00:00
1992-06-01 00:00:00
1992-07-13 00:00:00
1992-08-24 00:00:00
1992-10-05 00:00:00
1992-11-16 00:00:00
1992-12-28 00:00:00
1993-02-08 00:00:00
1993-03-22 00:00:00
1993-05-03 00:00:00
1993-06-14 00:00:00
1993-07-26 00:00:00
1993-09-06 00:00:00
1993-10-18 00:00:00
1993-11-29 00:00:00
1994-01-10 00:00:00
1994-02-21 00:00:00
1994-04-04 00:00:00
1994-05-16 00:00:00
1994-06-27 00:00:00
1994-08-08 00:00:00
1994-09-19 00:00:00
1994-10-31 00:00:00
1994-12-12 00:00:00
1995-01-23 00:00:00
1995-03-06 00:00:00
1995-04-17 00:00:00
1995-05-29 00:00:00
1995-07-10 00:00:00
1995-08-21 00:00:00
